# Arguments

In [1]:
import tensorflow as tf

model_load_path = 'vgg16-fined-tuned.h5'
model_save_path = './'
model_name = 'vgg16-fined-tuned.pb'

# Load model

In [2]:
import tensorflow as tf
print(tf.__version__)


model = tf.keras.models.load_model(
    model_load_path,
    custom_objects=None,
    compile=False
)

1.13.1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Get input and output names

In [3]:
layer_names=[layer.name for layer in model.layers]
print(layer_names)


['input_1', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool', 'flatten', 'dense_1', 'dropout_1', 'dense_2']


In [4]:
input_node_names = ['input_1']
output_node_name = 'dense_2'

# Convert model to tensorflow

In [5]:
# from keras import backend as K
# import tensorflow as tf

# def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
#     """
#     Freezes the state of a session into a pruned computation graph.

#     Creates a new computation graph where variable nodes are replaced by
#     constants taking their current value in the session. The new graph will be
#     pruned so subgraphs that are not necessary to compute the requested
#     outputs are removed.
#     @param session The TensorFlow session to be frozen.
#     @param keep_var_names A list of variable names that should not be frozen,
#                           or None to freeze all the variables in the graph.
#     @param output_names Names of the relevant graph outputs.
#     @param clear_devices Remove the device directives from the graph for better portability.
#     @return The frozen graph definition.
#     """
#     from tensorflow.python.framework.graph_util import convert_variables_to_constants
#     graph = session.graph
#     with graph.as_default():
#         freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
#         output_names = output_names or []
#         output_names += [v.op.name for v in tf.global_variables()]
#         # Graph -> GraphDef ProtoBuf
#         input_graph_def = graph.as_graph_def()
#         if clear_devices:
#             for node in input_graph_def.node:
#                 node.device = ""
#         frozen_graph = convert_variables_to_constants(session, input_graph_def,
#                                                       output_names, freeze_var_names)
#         return frozen_graph




In [6]:
# import keras
# session = keras.backend.get_session()
# init = tf.global_variables_initializer()
# session.run(init)

# frozen_graph = freeze_session(K.get_session(),
#                               output_names=[out.op.name for out in model.outputs])

# tf.train.write_graph(frozen_graph, "model", model_name, as_text=False)

# keras_to_tensorflow(model, output_dir=model_save_path, model_name=model_name)
# print('MODEL SAVED')

# Export model

In [7]:
import os
import os.path as path
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

from tensorflow.examples.tutorials.mnist import input_data

def export_model(saver, model, MODEL_NAME, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', saver, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    # Resave weights in format suitable for android
    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

Using TensorFlow backend.


In [8]:
import keras
session = keras.backend.get_session()
init = tf.global_variables_initializer()
session.run(init)

export_model(tf.train.Saver(), model, model_name, input_node_names, output_node_name)

Instructions for updating:
Use tf.gfile.GFile.


TypeError: Expected binary or unicode string, got <tensorflow.python.training.saver.Saver object at 0x633e79ba8>